In [1]:
# Install necessary packages
!pip install langchain-experimental nltk fastapi uvicorn nest-asyncio pyngrok openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0

In [2]:
# Import necessary packages
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool
from langchain_experimental.tools import PythonREPLTool
from nltk import sent_tokenize
import nltk
from typing import List, Any
import uvicorn

In [3]:
# Add your ngrok authtoken here
authtoken = "2k3UjxMzMMUpUMHR8Seg6mnlrNw_4gxn2JVstFDkMze6JqcAD"
ngrok.set_auth_token(authtoken)

# Download NLTK data for sentence tokenization
nltk.download('punkt')

# Apply asyncio patch to run FastAPI in Jupyter-like environments
nest_asyncio.apply()

# PlanAgent Class
class PlanAgent:
    def __init__(self):
        pass

    def split_task(self, user_query: str) -> List[str]:
        """
        Split the user query into multiple sub-tasks.
        """
        sub_tasks = sent_tokenize(user_query)
        return [task.strip() for task in sub_tasks if task.strip()]

    def refine_tasks(self, tasks: List[str], feedback: str) -> List[str]:
        """
        Refine tasks based on feedback.
        """
        refined_tasks = [task for task in tasks if "delete" not in feedback]
        refined_tasks.append("New task added based on feedback")
        return refined_tasks

# ToolAgent Class
class ToolAgent:
    def __init__(self, tools: List[Tool]):
        self.tools = tools

    def solve_task(self, task: str) -> Any:
        task = task.lower().strip()

        if "calculate" in task:
            # Example: Using PythonREPLTool to calculate expressions
            try:
                expression = task.split("calculate")[1].strip()
                result = self.tools[0].run(f"print({expression})").strip()

                try:
                    float_result = float(result)
                    if float_result.is_integer():
                        result = str(int(float_result))
                    else:
                        result = str(float_result)
                except ValueError:
                    result = "Invalid calculation result."
            except Exception as e:
                result = f"Error in calculation: {str(e)}"
        elif "check the status" in task:
            result = self.handle_status_check(task)
        elif "send email" in task:
            result = self.handle_email(task)
        else:
            result = f"Task '{task}' is not recognized and needs a more specific handler."
        return result

    def handle_status_check(self, task: str) -> str:
        return "Status check completed: All systems are operational."

    def handle_email(self, task: str) -> str:
        return "Email has been sent successfully."

# Feedback function
def provide_feedback(result: Any) -> str:
    if "error" in result.lower():
        return "modify task"
    else:
        return "task completed"

# Initialize PythonREPLTool and ToolAgent
python_tool = PythonREPLTool()
tool_agent = ToolAgent([python_tool])

# Initialize PlanAgent
plan_agent = PlanAgent()

# Main workflow
def process_query(user_query: str):
    sub_tasks = plan_agent.split_task(user_query)
    for task in sub_tasks:
        result = tool_agent.solve_task(task)
        feedback = provide_feedback(result)

        if feedback == "modify task":
            sub_tasks = plan_agent.refine_tasks(sub_tasks, feedback)
        else:
            print(f"Task '{task}' completed successfully: {result}")

# FastAPI Integration
app = FastAPI()

# Define input model for the FastAPI API
class QueryModel(BaseModel):
    query: str

# PlanAgent for FastAPI: Break user query into sub-tasks
def plan_agent_api(user_query: str):
    return plan_agent.split_task(user_query)

# ToolAgent for FastAPI: Solve sub-tasks
def tool_agent_api(sub_task: str):
    return tool_agent.solve_task(sub_task)

# Reflection and feedback in the FastAPI workflow
def feedback_and_reflection_api(sub_task_result: str):
    return provide_feedback(sub_task_result)

# Outer loop to handle sub-task refinement and execution
def refinement_loop_api(tasks: List[str]):
    results = []
    for sub_task in tasks:
        result = tool_agent_api(sub_task)
        results.append(result)

        feedback = feedback_and_reflection_api(result)
        if feedback == "modify task":
            tasks = plan_agent.refine_tasks(tasks, feedback)
    return results

# FastAPI endpoint to solve the query
@app.post("/solve")
def solve_query(query: QueryModel):
    tasks = plan_agent_api(query.query)
    solved_tasks = refinement_loop_api(tasks)
    return {"query": query.query, "tasks": solved_tasks}

# Create a public URL using ngrok for accessing the FastAPI app
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI app with Uvicorn
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:     Started server process [437]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://1bf9-35-245-164-51.ngrok-free.app" -> "http://localhost:8000"
INFO:     2406:7400:98:994c:8533:a5ec:4c26:34eb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2406:7400:98:994c:8533:a5ec:4c26:34eb:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     2406:7400:98:994c:8533:a5ec:4c26:34eb:0 - "POST /solve HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [437]
